# Modelling of a generic neutron pulse

This notebook presents a general model of a neutron source,
with an optional long-pulsed time structure [1] and an energy distribution of moderated and under-moderated neutrons [2].
This model is implemented in the **Source_custom**
[component](https://github.com/mccode-dev/McCode/pull/1911)
for the [McStas](https://mcstas.org/) Monte Carlo simulation package.
Finally, this notebook also provides a tool to manually fit data from real neutron sources to this model.

Import all necessary packages to load the interactive models:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets
from aton import phys
dataset = None

## Time structure

Exponential kinetics are usually modelled as the sum of one or two exponentials.
For an ascending curve over time with a decay factor $\tau$:
$$P_{\text{ascend}}(t)=A_{0}+\sum_{i}A_{i}\left(1-\exp\left(-\frac{t}{\tau_1}\right)\right)$$

And the decay curve would follow:
$$P_{\text{decay}}(t)=\sum_{i}A_{i}\cdot\exp\left({-\frac{t}{\tau_2}}\right)$$

With $A_i$ as the neutron flux, the time structure of a neutron long pulse according to [1] follows:
$$P_{t<t_p}=1-\exp\left(-\frac{t}{\tau/n}\right)$$
$$P_{t>tp}=\exp\left(-\frac{t-t_p}{\tau}\right)-\exp\left(-\frac{t}{\tau/n}\right)$$

with $t_p$ the pulse length, and $n$ the ratio between time decay and ascend constants.

The pulse is normalised by dividing it by its integral, so that the area under the curve is equal to 1.
See the [annex](#annex-integral-of-the-pulse-profile) for more details.

You can interact with this model running the cell below.

In [ ]:
def pulse_carpenter(t, tau, t_pulse, n) -> np.ndarray:
    result = np.zeros_like(t)
    mask = t <= t_pulse
    integral = t_pulse + tau - tau/n
    # t <= t_pulse
    result[mask] = (1 - np.exp(-t[mask] / (tau/n))) / integral
    # t > t_pulse
    result[~mask] = (np.exp(-(t[~mask]-t_pulse)/tau) - np.exp(-t[~mask]/(tau))) / integral
    print(f'Integral: {np.trapezoid(result, t):.2}')
    return result

def interactive_pulse_carpenter(t_pulse, tau, n, log, xzoom):
    #plt.clf()
    tmax = t_pulse / xzoom
    fig, ax = plt.subplots()
    t = np.linspace(0, tmax, 100)
    i = pulse_carpenter(t, tau, t_pulse, n)
    ax.plot(t, i)
    ax.set_xlabel('Time / s')
    ax.set_ylabel('Intensity / a.u.')
    ax.set_xlim(0, tmax)
    if log:
        ax.set_yscale('log')
    plt.show()
    return None

t_pulse = ipywidgets.FloatSlider(value=0.0026, min=0.00001, max=0.01, step=1e-5, description='t_pulse', readout_format='.3')
tau     = ipywidgets.FloatSlider(value=0.0001, min=1e-5,  max=1e-3, step=1e-5, description='tau', readout_format='.4')
n       = ipywidgets.FloatLogSlider(value=1.0, base=10, min=-1,  max=1, step=0.01, description='n', readout_format='.2')
log     = ipywidgets.Checkbox(value=False, description='Log Scale')
xzoom   = ipywidgets.FloatLogSlider(value=0.5, base=10, min=-1, max=1, step=0.1, description='X Zoom', readout_format='.1')

ipywidgets.interact(interactive_pulse_carpenter, t_pulse=t_pulse, tau=tau, n=n, log=log, xzoom=xzoom)


## Energy distribution

### Moderated neutrons

The wavelength distribution from a neutron source should follow a Maxwellian distribution according to the moderator temperature [2]:
$$M(\lambda)=\frac{2a^2}{T^2\lambda^5}\exp\left(-\frac{a}{T\lambda^{2}}\right)$$
$$a=\left(\frac{h^2}{2m_{N}k_{B}}\right)$$

Below is an interactive model for this purely Maxwellian distribution:


In [ ]:
def maxwell(lmbd, temperature):
    a = 949.29 / temperature
    M = 2.0 * a*a * np.exp(-a/(lmbd*lmbd)) / lmbd**5
    integral = f'Integral: {np.trapezoid(M, lmbd):.2}'
    return M, integral

def interactive_maxwell(temperature, xzoom):
    #plt.clf()
    fig, ax = plt.subplots()
    max_lmbd = np.sqrt(5000/temperature) / xzoom
    lmbd = np.linspace(0.001, max_lmbd, 100)
    T, integral = maxwell(lmbd, temperature)
    print(integral)
    ax.plot(lmbd, T)
    ax.set_xlabel('Wavelength / AA')
    ax.set_ylabel('Intensity / a.u.')
    ax.set_xlim(0.0, max_lmbd)
    plt.show()
    return None

temperature = ipywidgets.FloatSlider(value=300, min=1, max=1000, step=1, description='Temperature', readout_format='.4')
xzoom = ipywidgets.FloatLogSlider(value=1, base=10, min=-1, max=1, step=0.1, description='X Zoom', readout_format='.1')

ipywidgets.interact(interactive_maxwell, temperature=temperature, xzoom=xzoom)

### Under-moderated neutrons

Moderators are usually thin to avoid big flux losses. This leads to some under-moderated neutrons to escape with high energies.
These are modelled by a so-called *joining function* [2]:

$$M(\lambda)_{um}=\frac{1}{\lambda(1+\exp(\lambda\chi-\kappa))}$$

With $\chi$ as an experimental factor for the wavelength dependence of under-moderated neutrons,
and $\kappa$ as an experimental scaling factor for the flux of under-moderated neutrons.

Note that a simpler model of a joining function, $M(\lambda)_{um}=1/\lambda$, can be achieved with $\chi=0$ and $\kappa=\infty$.

The interactive model below considers only the under-moderated contributions:

In [ ]:
def joining_function(lmbd, chi, kappa):
    intensity =  1.0 / ((1.0 + np.exp(chi * lmbd - kappa)) * lmbd)
    integral = f'Integral: {np.trapezoid(intensity, lmbd):.2}'
    return intensity, integral

def interactive_joining_function(chi, kappa, xzoom, log):
    #plt.clf()
    fig, ax = plt.subplots()
    max_lmbd = 1.0 / xzoom
    lmbd = np.linspace(0.001, max_lmbd, 1000)
    J, integral = joining_function(lmbd, chi, kappa)
    print(integral)
    ax.plot(lmbd, J)
    ax.set_xlabel('Wavelength / AA')
    ax.set_ylabel('Intensity / a.u.')
    ax.set_xlim(0.0, max_lmbd)
    if log:
        ax.set_yscale('log')
    plt.show()
    return None

chi = ipywidgets.FloatSlider(value=2.5, min=-10, max=10, step=0.1, description='chi', readout_format='.2')
kappa = ipywidgets.FloatSlider(value=2.2, min=-10, max=10, step=0.1, description='kappa', readout_format='.2')
xzoom = ipywidgets.FloatLogSlider(value=0.5, base=10, min=-1, max=1, step=0.1, description='X Zoom', readout_format='.1')
log = ipywidgets.Checkbox(value=True, description='Log Scale')

ipywidgets.interact(interactive_joining_function, chi=chi, kappa=kappa, xzoom=xzoom, log=log)

### All contributions

The total energy distribution will be a combination of the moderated and unmoderated contributions, each with a particular flux $I_i$
$$M(\lambda)_{total} = I_{um} \cdot M(\lambda)_{um} + \sum_i I_i \cdot M(\lambda)_i$$

The model below considers all contributions, both from moderated and under-moderated contributions.
Up to three Maxwellian distributions for the moderated neutrons are considered.

The available input values are the same as those of the **Source_custom** McStas component (except for the plotting options).

In [ ]:
def spectra(lmbd, T1, I1, T2, I2, T3, I3, Ium, chi, kappa):
    maxwell_1, _ = maxwell(lmbd, T1)
    maxwell_2, _ = maxwell(lmbd, T2)
    maxwell_3, _ = maxwell(lmbd, T3)
    unmoderated, _ = joining_function(lmbd, chi, kappa)
    moderated = I1 * maxwell_1 + I2 * maxwell_2 + I3 * maxwell_3
    M = Ium * unmoderated + moderated
    max_maxwell = np.max(moderated)
    return M, max_maxwell

def interactive_spectra(T1, I1, T2, I2, T3, I3, Ium, chi, kappa, xzoom, yzoom, cutoff, flux_multiplier, plot_type):
    fig, ax = plt.subplots()
    max_lmbd = np.sqrt(6000/T1) / xzoom
    lmbd = np.linspace(cutoff, max_lmbd, 100)
    M, ymax = spectra(lmbd, T1, I1*flux_multiplier, T2, I2*flux_multiplier, T3, I3*flux_multiplier, Ium*flux_multiplier, chi, kappa)
    ymax = 1.1 * ymax / yzoom
    if plot_type == 'wavelength':
        ax.plot(lmbd, M)
        if dataset:
            ax.plot(dataset[0], dataset[1], marker='.', linestyle='')
        ax.set_xlim(0.0, max_lmbd)
        ax.set_ylim(0.0, ymax)
        ax.set_xlabel('Wavelength / $\\AA$')
        ax.set_ylabel('Intensity / a.u.')
    else:
        E = phys.AA_to_meV * phys.meV_to_eV / (lmbd**2)
        ax.plot(E, M)
        if dataset:
            dataset_E = phys.AA_to_meV * phys.meV_to_eV / (dataset[0]**2)
            ax.plot(dataset_E, dataset[1], marker='.', linestyle='')
        ax.set_xlabel('Energy / eV')
        ax.set_ylabel('Intensity / a.u.')
        plt.xscale('log')
        plt.yscale('log')
    return None

T1 = ipywidgets.FloatSlider(value=300, min=1, max=1000, step=1, description='T1', readout_format='.4')
I1 = ipywidgets.FloatSlider(value=1, min=0, max=1, step=0.001, description='I1', readout_format='.4')
T2 = ipywidgets.FloatSlider(value=60, min=1, max=1000, step=1, description='T2', readout_format='.4')
I2 = ipywidgets.FloatSlider(value=0, min=0, max=1, step=0.001, description='I2', readout_format='.4')
T3 = ipywidgets.FloatSlider(value=30, min=1, max=1000, step=1, description='T3', readout_format='.4')
I3 = ipywidgets.FloatSlider(value=0, min=0, max=1, step=0.001, description='I3', readout_format='.4')
Ium = ipywidgets.FloatSlider(value=0, min=0, max=1, step=0.001, description='I_um', readout_format='.4')
chi = ipywidgets.FloatSlider(value=0, min=-20, max=20, step=0.001, description='chi', readout_format='.4')
kappa = ipywidgets.FloatSlider(value=0, min=-20, max=20, step=0.001, description='kap', readout_format='.4')
xzoom = ipywidgets.FloatLogSlider(value=1, base=10, min=-1, max=1, step=0.01, description='X Zoom', readout_format='.3')
yzoom = ipywidgets.FloatLogSlider(value=1, base=10, min=-1, max=1, step=0.01, description='Y Zoom', readout_format='.3')
cutoff = ipywidgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Low cutoff', readout_format='.3')
flux_multiplier = ipywidgets.FloatLogSlider(value=-1, base=10, min=-2, max=15, step=0.01, description='Flux multiplier', readout_format='.2')
plot_type = ipywidgets.RadioButtons(options=['wavelength', 'energy'], value='wavelength', description='Plot type')

ipywidgets.interact(interactive_spectra, T1=T1, I1=I1, T2=T2, I2=I2, T3=T3, I3=I3, Ium=Ium, chi=chi, kappa=kappa, xzoom=xzoom, yzoom=yzoom, cutoff=cutoff, flux_multiplier=flux_multiplier, plot_type=plot_type)

## Fitting actual data from real neutron sources

You may want to fit experimental spectra from real sources, in order to use them in McStas with the **Source_custom** component.
You can do so with [SciPy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).
However this can get messy, since there are quite a few variables to fit.
As a first approximation, it can be useful to fit the dataset by hand, before refining it with the algorythm.

You can load your own dataset in the cell below.
Then run again the cell above with the full model, and your dataset will be displayed alongside the model.
Remember to use the flux multiplier and zoom sliders!

Once you know the parameters of your source, you should try to reproduce it on McStas.
Note that you will probably need to increase the flux while keeping the ratios constant.
Remember to normalise the intensities (by binning, etc.) if necessary!

In [ ]:
filename = './data/RANS1_ma2021_2m_PE_4cm.csv'

# Your data might need a bit of processing, e.g. it might be in eV instead of AA... Fix it here!
data = pd.read_csv(filename, comment='#', header=None)
intensity  = data.iloc[:, 1].values
energy     = data.iloc[:, 0].values
wavelength = np.sqrt(phys.AA_to_meV / (energy * phys.eV_to_meV))
dataset = (wavelength, intensity)
# data = None  # To remove the dataset

print(data)

## References

- [1] J. M. Carpenter and W. B. Yelon, Methods in Experimental Physics 23, p. 127, Chapter 2, Neutron Sources (1986).
- [2] J. M. Carpenter et al, Nuclear Instruments and Methods in Physics Research Section A, 234, 542–551 (1985).

## Annex: integral of the pulse profile

The pulse profile is divided by its integral in order to normalise it:

$$
\int_0^{\infty} P(t) dt = \int_0^{t_p} P(t) dt + \int_{t_p}^{\infty} P(t) dt
$$

$$
\int_0^{t_p}\left(1-\exp\left(-\frac{tn}{\tau}\right)\right)dt =
\left|t+\frac{\tau\exp\left(-\frac{tn}{\tau}\right)}{n} + C\right|_0^{t_p} =
\frac{\tau\exp\left(-\frac{nt_p}{\tau}\right)}{n} + t_p - \frac{\tau}{n}
$$

$$
\int_{t_p}^{\infty}\left(\exp\left(-\frac{t-tp}{\tau}\right)-\exp\left(-\frac{tn}{\tau}\right)\right)dt =
\left|\frac{\tau\exp\left(-\frac{tn}{\tau}\right)}{n} - \tau\exp\left(\frac{t_p-t}{\tau}\right) + C\right|_{t_p}^{\infty} = 
-\frac{\tau\exp\left(-\frac{nt_p}{\tau}\right)}{n} + \tau
$$

Therefore the integral of the whole pulse is:
$$
\int_0^{\infty} P(t) dt = t_p + \tau - \frac{\tau}{n}
$$
